In [65]:
#!pip install rtmlib==0.0.13
#!pip install opencv-python==4.11.0.86
#!pip install librosa==0.11.0
#!pip install Cython==3.0.12
# Microsoft Visual C++ 14.0 or greater is required.
# Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# The pip version of madmon is outdated and the github repo has many changes recently
#!pip uninstall -y madmom
#!conda install git+https://github.com/CPJKU/madmom.git
#!cd madmom && python setup.py develop --user
#!pip install pyaudio==0.2.14
#!pip install beatnet==1.1.1
#!pip install soundfile==0.13.1
# Install FFmpeg in Windows
# Source: https://phoenixnap.com/kb/ffmpeg-windows
#!pip cache purge
#!pip install moviepy==2.1.1
#!pip install onnxruntime==1.21.1

In [13]:
import os

move_filename = '1841f9d_1'
moves_filepath = os.path.join('outputs', 'moves.json')

In [12]:
!ffmpeg -i inputs/videos/1841f9d_1.mp4 -ab 160k -ac 2 -ar 44100 -vn processed/videos/1841f9d_1.wav

^C


In [15]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
audio_filepath = os.path.join('processed', 'videos', f'{move_filename}.wav')
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(audio_filepath)
beats

C:\Users\user\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


array([[0.24, 1.  ],
       [0.7 , 2.  ],
       [1.2 , 1.  ],
       [1.66, 2.  ],
       [2.18, 1.  ],
       [2.66, 2.  ],
       [3.14, 1.  ],
       [3.62, 2.  ],
       [4.1 , 1.  ],
       [4.6 , 2.  ],
       [5.08, 1.  ],
       [5.58, 2.  ],
       [6.08, 1.  ],
       [6.56, 2.  ],
       [7.04, 1.  ],
       [7.52, 2.  ],
       [8.  , 1.  ]])

In [17]:
# 9da1eb8 = 4
# 5db13cd = 1
# 5d80c9a = 1
# 022dccf = - 1

In [17]:
offset = 0
# Fix the beat numbers
for index in range(len(beats)):
    beats[index][1] = int((index + offset) % 8)
beats

array([[0.24, 0.  ],
       [0.7 , 1.  ],
       [1.2 , 2.  ],
       [1.66, 3.  ],
       [2.18, 4.  ],
       [2.66, 5.  ],
       [3.14, 6.  ],
       [3.62, 7.  ],
       [4.1 , 0.  ],
       [4.6 , 1.  ],
       [5.08, 2.  ],
       [5.58, 3.  ],
       [6.08, 4.  ],
       [6.56, 5.  ],
       [7.04, 6.  ],
       [7.52, 7.  ],
       [8.  , 0.  ]])

In [59]:
'''
offset = 0
beat_start = 0.76
beat_end = 8.58
num_beats = 17
beat_delta = (beat_end - beat_start) / (num_beats - 1)
beats = [[round(beat_start + x * beat_delta, 2), (x + offset) % 8] for x in range(num_beats)]
beats
'''

[[0.76, 0],
 [1.25, 1],
 [1.74, 2],
 [2.23, 3],
 [2.71, 4],
 [3.2, 5],
 [3.69, 6],
 [4.18, 7],
 [4.67, 0],
 [5.16, 1],
 [5.65, 2],
 [6.14, 3],
 [6.62, 4],
 [7.11, 5],
 [7.6, 6],
 [8.09, 7],
 [8.58, 0]]

In [11]:
'''
# Fix the beat numbers
for index in range(12, 16):
    beats[index][1] = beats[index][1] - 4
beats
'''

'\n# Fix the beat numbers\nfor index in range(12, 16):\n    beats[index][1] = beats[index][1] - 4\nbeats\n'

In [19]:
from moviepy import VideoFileClip, TextClip, CompositeVideoClip

video_file = f"inputs/videos/{move_filename}.mp4"
output_file = f"processed/videos/{move_filename}_counts.mp4"


# Load the video
video_clip = VideoFileClip(video_file)

start_counting = True
text_clips = [video_clip]

for index, x in enumerate(beats):
    start_time = x[0]
    end_time = beats[index + 1][0] if index + 1 < len(beats) else x[0]

    start_counting = start_counting or x[1] == 1

    if start_counting:
        # Create the text clip
        text_clip = TextClip(font='arial.ttf', text=f'{int(x[1] + 1)}', font_size=70, color='white', duration=end_time - start_time)
        
        # Set the duration and position of the text clip
        text_clip = text_clip.with_position(('center', 'bottom'))
        
        # Set the start time of the text clip
        text_clip = text_clip.with_start(start_time)

        text_clips.append(text_clip)
        
# Composite the text clip on the video
video_clip = CompositeVideoClip(text_clips)

# Write the output video file
video_clip.write_videofile(output_file, fps=30, codec='libx264')

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 5468, 'fps': 50.12, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.100 libx264'}}], 'input_number': 0}], 'duration': 8.02, 'bitrate': 5594, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 44100, 'audio_bitrate': 127, 'default_video_input_number': 0, 'default_video_stream_number': 1, 'video_codec_n

MoviePy - Done.
MoviePy - Writing video processed/videos/1841f9d_1_counts.mp4



MoviePy - Done !
MoviePy - video ready processed/videos/1841f9d_1_counts.mp4


In [46]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# After verifying that the counts are Okay, I can save the counts
data[move_filename] = []
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    data[move_filename].append({
        'section': section, 
        'count': count,
        'time': x[0],
        'leader_keypoints': [],
        'follower_keypoints': [],
        'offset': offset
    })
    if count == 8:
        section += 1

with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1


In [38]:
import cv2

start_counting = False
section = 1
for index, x in enumerate(beats):
    count = int(x[1] + 1)
    print(count)
    vidcap = cv2.VideoCapture(video_file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, int(x[0] * 1000))      # just cue to position
    success, image = vidcap.read()
    if success:
        cv2.imwrite(os.path.join('processed', 'images', f"{move_filename}_section_{section}_count_{count}.jpg"), image)     # save frame as JPEG file
    else:
        print(f'Error in index: {index}')
        # data[move_filename].pop(index)
    # Release resources
    vidcap.release()
    cv2.destroyAllWindows()

    if count == 8 or (count == 4 and index < len(beats) - 2 and beats[index + 1][1] == 0):
        section += 1

8
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:929: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [30]:
import numpy as np
import cv2
import os
from rtmlib import Body, draw_skeleton
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

device = 'cpu'
backend = 'onnxruntime'  # opencv, onnxruntime, openvino

cap = cv2.VideoCapture(0)

openpose_skeleton = False  # True for openpose-style, False for mmpose-style

body = Body(
    pose='rtmo',
    to_openpose=openpose_skeleton,
    mode='performance',  # balanced, performance, lightweight
    backend=backend,
    device=device
)
# TODO: Nowadays, I do this by hand. We need to automate it.
leader_detected_first = [
    False,
    True,
    False,
    True,
    True,
    False,
    True,
    False,
    True,
    True,
    False,
    False,
    False,
    False,
    True,
    True,
    True
]
leader_detected_first = [False for index in range(len(beats))]
img_index = 0

for filename in os.listdir(os.path.join('processed', 'images')):
    # check if the image ends with png
    if (filename.startswith(move_filename)):
        frame = cv2.imread(os.path.join('processed', 'images', filename), cv2.IMREAD_COLOR)
        keypoints, scores = body(frame)

        if leader_detected_first[img_index]:
            data[move_filename][img_index]["leader_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["follower_keypoints"] = keypoints[1].tolist()
        elif np.sum(keypoints) > 0:
            data[move_filename][img_index]["follower_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["leader_keypoints"] = keypoints[1].tolist()
        print(filename)
        print(keypoints.shape)
        print(np.sum(keypoints))
        print(keypoints[0])

        img_show = frame.copy()
        img_show = draw_skeleton(
            img_show,
            keypoints,
            #np.array([keypoints[0]]),
            scores,
            openpose_skeleton=openpose_skeleton,
            kpt_thr=0.3,
            line_width=2
        )

        cv2.imwrite(os.path.join('processed', 'keypoints', filename), img_show)
        img_index += 1

# Override the data with the keypoints
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

load C:\Users\user\.cache\rtmlib\hub\checkpoints\rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.onnx with onnxruntime backend
1841f9d_1_section_1_count_8.jpg
(1, 17, 2)
14860.532
[[572.6382  159.25513]
 [583.3111  150.37009]
 [562.34656 148.47055]
 [596.1051  161.51614]
 [544.71063 156.6496 ]
 [610.8037  235.42099]
 [517.19    221.38155]
 [617.2791  326.221  ]
 [459.42847 289.6308 ]
 [673.4911  301.2173 ]
 [431.3399  229.87305]
 [590.16754 384.24118]
 [522.02893 379.42407]
 [632.74567 499.22003]
 [464.86594 502.4087 ]
 [671.9671  630.59863]
 [405.97043 628.2439 ]]
1841f9d_1_section_2_count_1.jpg
(1, 17, 2)
14942.77
[[580.4832  152.26785]
 [592.0322  142.57169]
 [571.1813  141.93353]
 [607.9581  150.76276]
 [557.2615  148.19855]
 [623.85114 220.48848]
 [535.2545  214.95619]
 [673.53827 280.77438]
 [474.8368  264.8934 ]
 [735.97107 268.6872 ]
 [474.29413 197.63629]
 [596.51245 361.0144 ]
 [537.03375 358.08282]
 [584.1471  492.9669 ]
 [533.44727 492.32843]
 [590.1401  621.1842 ]
 [542

In [71]:
import numpy as np

def normalize_keypoints_by_bbox_height(keypoints):
    """
    Normalize keypoints so that:
    - The top-left of the bounding box is (0, 0)
    - The bounding box height is 1
    """
    kp = np.array(keypoints)
    
    min_x = np.min(kp[:, 0])
    min_y = np.min(kp[:, 1])
    max_y = np.max(kp[:, 1])
    
    bbox_height = max_y - min_y
    if bbox_height < 1e-5:
        raise ValueError("Bounding box height too small for normalization.")
    
    # Translate to origin and scale
    kp_normalized = (kp - [min_x, min_y]) / bbox_height
    return kp_normalized, bbox_height


def compute_normalized_embeddings(leader_kp, follower_kp):
    leader_kp = np.array(leader_kp)
    follower_kp = np.array(follower_kp)

    # Normalize each person's keypoints by their own bbox height
    norm_leader = np.array([])
    norm_follower = np.array([])
    if np.sum(leader_kp) > 0:
        norm_leader, leader_height = normalize_keypoints_by_bbox_height(leader_kp)
    if np.sum(follower_kp) > 0:
        norm_follower, _ = normalize_keypoints_by_bbox_height(follower_kp)

    rel_vector = np.array([])
    if np.sum(leader_kp) > 0 and np.sum(follower_kp) > 0:
        # Get original centers (before normalization)
        center_leader = np.mean(leader_kp, axis=0)
        center_follower = np.mean(follower_kp, axis=0)
    
        # Compute relative position vector in leader's height units
        rel_vector = (center_follower - center_leader) / leader_height

    return norm_leader.flatten(), norm_follower.flatten(), rel_vector

In [73]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

for index in range(len(data[move_filename])):
    kp_leader = data[move_filename][index]['leader_keypoints']
    kp_follower = data[move_filename][index]['follower_keypoints']
    leader_embedding, follower_embedding, distance_embedding = compute_normalized_embeddings(kp_leader, kp_follower)
    data[move_filename][index]['leader_embedding'] = leader_embedding.tolist()
    data[move_filename][index]['follower_embedding'] = follower_embedding.tolist()
    data[move_filename][index]['distance_embedding'] = distance_embedding.tolist()

# Override the data with the new embeddings
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [75]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# Add musicality (TODO: This is manual)
musicality = [
    '-',
    '-',
    '-',
    '-',
    '~4',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '*',
    '-',
    '-',
    '-',
    '-'
]
for index in range(len(data[move_filename])):
    data[move_filename][index]['musicality'] = musicality[index]

# Override the data with musicality
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [1]:
import random
import numpy as np
import json
import os

moves_filepath = os.path.join('outputs', 'moves.json')
# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    moves_data = json.load(file)

# -------------------------
# 1. Build embeddings
# -------------------------
def build_embedding(move_item):
    """Return a 34+34+2 = 70 sized embedding"""

    l = move_item.get("leader_embedding", [])
    f = move_item.get("follower_embedding", [])
    d = move_item.get("distance_embedding", [])

    # pad or slice to fixed sizes
    l = (l + [0]*34)[:34]
    f = (f + [0]*34)[:34]
    d = (d + [0]*2)[:2]

    return np.array(l + f + d, dtype=float)

# -------------------------
# 2. Cosine Similarity
# -------------------------
def cosine_sim(a, b):
    if np.all(a == 0) or np.all(b == 0):
        return 0
    return np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b))

embedding_map = {x + 1: [] for x in range(8)}
for move_id in moves_data:
    current_move = moves_data[move_id]
    for index, move_item in enumerate(current_move):
        current_count = move_item['count']
        embedding_map[move_item['count']].append({
            'move_id': move_id,
            'index': index,
            'embedding': build_embedding(move_item)
        })

for count, items in embedding_map.items():
    for index, item in enumerate(items):
        current_embedding = item['embedding']
        embedding_candidates = [x['embedding'] for x in items]
        similarities = [(cosine_sim(current_embedding, embedding), idx) for idx, embedding in enumerate(embedding_candidates)]
        similarities = sorted(similarities, key=lambda x: x[0], reverse=True)
        move_id = item['move_id']
        move_index = item['index']
        moves_data[move_id][move_index]['connections'] =[]
        for (similarity, candidate_index) in similarities:
            moves_data[move_id][move_index]['connections'].append({
                'move_id': items[candidate_index]['move_id'],
                'index': items[candidate_index]['index'],
                'similarity': float(similarity)
            })

# Override the data with moves candidates per step
with open(moves_filepath, 'w') as fp:
    json.dump(moves_data, fp)

In [25]:
import json
import os

moves_filepath = os.path.join('outputs', 'moves.json')
# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    moves_data = json.load(file)

for move_id, data in moves_data.items():
    print(f'{move_id}: {len(data)}')
#moves_data['1841f9d'][8]
#moves_data['5db13cd'][3]

5db13cd: 18
9da1eb8: 19
5d80c9a: 21
8440066: 16
bfb0179: 18
ea90f11: 17
5b961d4: 9
48230c1: 13
29aec7e: 17
fc95277: 15
1841f9d: 16
7254257: 17
acdbcc0: 24
4070a1d: 9
0cced52: 16
3e7415f: 20
a0cfff7: 16
022dccf: 18
5365862: 28
e3cbffd: 17


In [23]:
#del moves_data['1841f9d_1']
#moves_data['1841f9d'].pop()
with open(moves_filepath, 'w') as fp:
    json.dump(moves_data, fp)

In [41]:
#moves_data['5db13cd'][17]
moves_data['1841f9d'][0]

{'section': 1,
 'count': 1,
 'time': 0.24,
 'leader_keypoints': [],
 'follower_keypoints': [[572.63818359375, 159.255126953125],
  [583.3110961914062, 150.37008666992188],
  [562.3465576171875, 148.47055053710938],
  [596.1051025390625, 161.51614379882812],
  [544.7106323242188, 156.64959716796875],
  [610.8037109375, 235.42098999023438],
  [517.1900024414062, 221.3815460205078],
  [617.2791137695312, 326.22100830078125],
  [459.428466796875, 289.63079833984375],
  [673.4910888671875, 301.21728515625],
  [431.33990478515625, 229.873046875],
  [590.1675415039062, 384.2411804199219],
  [522.0289306640625, 379.424072265625],
  [632.7456665039062, 499.22003173828125],
  [464.8659362792969, 502.40869140625],
  [671.9671020507812, 630.5986328125],
  [405.9704284667969, 628.243896484375]],
 'leader_embedding': [],
 'follower_embedding': [0.3456918633330154,
  0.022368695814436082,
  0.3678289530194027,
  0.003939899380778008,
  0.3243456145768935,
  0.0,
  0.39436548307854236,
  0.02705835594